In [1]:
import torch
import hess
import math
import numpy as np
from hess.nets import Transformer
from hess.eigs import hessian_eigenpairs

In [2]:
nx = 500
train_x = torch.linspace(0, 10, nx).unsqueeze(-1)
train_y = torch.sin(train_x * math.pi) + torch.sin(train_x)

In [3]:
optimus = Transformer(train_x, train_y, n_hidden=2, hidden_size=10,
                     activation=torch.nn.Tanh())

In [4]:
trained_pars = torch.load("../hess/saved-models/toy_regression.pt", map_location="cpu")

In [5]:
hessian = torch.load("../hess/saved-models/toy_hessian.pt", map_location="cpu")

In [6]:
e_val, e_vec = np.linalg.eig(hessian)
idx = e_val.argsort()[::-1]   
e_val = torch.FloatTensor(e_val[idx].real)
e_vec = torch.FloatTensor(e_vec[:,idx].real)

## Try the lanczos stuff

In [7]:
test_vals, test_vecs = hessian_eigenpairs(optimus.net,
                                        criterion=torch.nn.MSELoss, inputs=train_x,
                                         targets=train_y)

torch.Size([500, 1])
torch.Size([500, 1])


RuntimeError: bool value of Tensor with more than one value is ambiguous

In [13]:
crit = torch.nn.MSELoss()
train_x.shape

torch.Size([500, 1])

In [14]:
crit(train_y, train_x)

tensor(33.4606)